In [1]:
#Input DataFrame have 4 column Name Gender Subject and Marks
#add a column Performance
#update the column value as below
# col : if Value : Then Value
# Gender : M : Male
# Performance : value > 90 => Great Performance
#               90 > value > 70 => Good Performance
#               70 > value > 50 => Satisfactory Performance
#               50 > value => Needs Improvment

#don't use when() function 

In [2]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import functions as f
from pyspark.sql.functions import col, lit
from pyspark.sql import DataFrame

spark=SparkSession.builder.appName("Spark UDF Practise").getOrCreate()
#create DataFrame with movie CSV Dat
rawDF=spark.read.format("CSV").option("delimiter",";").load("/FileStore/tables/student_marks_subj.csv")
DFwithSchema=rawDF.toDF("Name", "Gender", "Subject", "Marks")
DFwithSchema.printSchema()

root
-- Name: string (nullable = true)
-- Gender: string (nullable = true)
-- Subject: string (nullable = true)
-- Marks: string (nullable = true)

In [3]:
def updateColumn(colName, value):
  if(colName == "Gender"):
    if(str(value)=="M"):
      return "Male"
    else:
      return "Female"
  elif(colName == "Performance"):
    if(int(value)>90):
      return "Great Performance"
    elif(70<int(value)<90):
      return "Good Performance"
    elif(50<int(value)<70):
      return "Satisfactory Performance"
    else:
      return "Needs Improvment"
  else:
    return str(colName)+"Column=="+str(value)


In [4]:
from pyspark.sql.functions import udf

modifyData = udf(updateColumn)
performance_DF=DFwithSchema.withColumn("Performance",lit(None))
performance_DF.schema

Out[ 104 ]: StructType(List(StructField(Name,StringType,true),StructField(Gender,StringType,true),StructField(Subject,StringType,true),StructField(Marks,StringType,true),StructField(Performance,NullType,true)))

In [5]:
performance_DF=performance_DF.select('Name', modifyData(lit('Gender'), col("Gender")).alias("Gender"), 'Subject', 'Marks',  modifyData(lit('Performance'), col("Marks")).alias("Performance"))

In [6]:
performance_DF.show(100)

+-------+------+---------+-----+--------------------+
 Name|Gender| Subject|Marks| Performance|
+-------+------+---------+-----+--------------------+
 John| Male| Maths| 46| Needs Improvment|
 John| Male| Physics| 65|Satisfactory Perf...|
 John| Male|Chemistry| 45| Needs Improvment|
 John| Male| English| 32| Needs Improvment|
 John| Male| Biology| 32| Needs Improvment|
 John| Male|Economics| 65|Satisfactory Perf...|
 Suresh| Male| Maths| 95| Great Performance|
 Suresh| Male| Physics| 45| Needs Improvment|
 Suresh| Male|Chemistry| 67|Satisfactory Perf...|
 Suresh| Male| English| 86| Good Performance|
 Suresh| Male| Biology| 76| Good Performance|
 Suresh| Male|Economics| 86| Good Performance|
 Ramesh| Male| Maths| 45| Needs Improvment|
 Ramesh| Male| Physics| 97| Great Performance|
 Ramesh| Male|Chemistry| 57|Satisfactory Perf...|
 Ramesh| Male| English| 75| Good Performance|
 Ramesh| Male| Biology| 87| Good Performance|
 Ramesh| Male|Economics| 57|Satisfactory Perf...|
 Ron| Male| Maths| 95| Great Performance|
 Ron| Male| Physics| 97| Great Performance|
 Ron| Male|Chemistry| 85| Good Performance|
 Ron| Male| English| 49| Needs Improvment|
 Ron| Male| Biology| 79| Good Performance|
 Ron| Male|Economics| 61|Satisfactory Perf...|
Charlie| Male| Maths| 53|Satisfactory Perf...|
Charlie| Male| Physics| 69|Satisfactory Perf...|
Charlie| Male|Chemistry| 93| Great Performance|
Charlie| Male| English| 85| Good Performance|
Charlie| Male| Biology| 74| Good Performance|
Charlie| Male|Economics| 91| Great Performance|
 Alan| Male| Maths| 55|Satisfactory Perf...|
 Alan| Male| Physics| 60|Satisfactory Perf...|
 Alan| Male|Chemistry| 84| Good Performance|
 Alan| Male| English| 51|Satisfactory Perf...|
 Alan| Male| Biology| 76| Good Performance|
 Alan| Male|Economics| 73| Good Performance|
 Jake| Male| Maths| 89| Good Performance|
 Jake| Male| Physics| 78| Good Performance|
 Jake| Male|Chemistry| 98| Great Performance|
 Jake| Male| English| 98| Great Performance|
 Jake| Male| Biology| 98| Great Performance|
 Jake| Male|Economics| 66|Satisfactory Perf...|
 Rose|Female| Maths| 88| Good Performance|
 Rose|Female| Physics| 92| Great Performance|
 Rose|Female|Chemistry| 52|Satisfactory Perf...|
 Rose|Female| English| 87| Good Performance|
 Rose|Female| Biology| 97| Great Performance|
 Rose|Female|Economics| 87| Good Performance|
 Mia|Female| Maths| 87| Good Performance|
 Mia|Female| Physics| 57|Satisfactory Perf...|
 Mia|Female|Chemistry| 89| Good Performance|
 Mia|Female| English| 62|Satisfactory Perf...|
 Mia|Female| Biology| 82| Good Performance|
 Mia|Female|Economics| 64|Satisfactory Perf...|
 Berta|Female| Maths| 59|Satisfactory Perf...|
 Berta|Female| Physics| 46| Needs Improvment|
 Berta|Female|Chemistry| 64|Satisfactory Perf...|
 Berta|Female| English| 61|Satisfactory Perf...|
 Berta|Female| Biology| 66|Satisfactory Perf...|
 Berta|Female|Economics| 47| Needs Improvment|
 Judith|Female| Maths| 84| Good Performance|
 Judith|Female| Physics| 46| Needs Improvment|
 Judith|Female|Chemistry| 79| Good Performance|
 Judith|Female| English| 45| Needs Improvment|
 Judith|Female| Biology| 52|Satisfactory Perf...|
 Judith|Female|Economics| 52|Satisfactory Perf...|
 Ross| Male| Math| 80| Good Performance|
 Ross| Male| Physics| 80| Good Performance|
 Ross| Male|Chemistry| 80| Good Performance|
 Ross| Male| English| 80| Good Performance|
 Ross| Male| Biology| 80| Good Performance|
 Ross| Male|Economics| 80| Good Performance|
+-------+------+---------+-----+--------------------+